In [1]:
import redis
import json

# Define connection variables
host = 'localhost'
port =  6379
password = None #'password'

# Connect to Redis
r = redis.Redis(host=host, port=port, password=password, decode_responses=True)
print('Connected to Redis')

r.flushdb()

Connected to Redis


True

# User Behavior Analysis for New Features

## Description of the use case
This use case involves analyzing user behavior to understand how new banking features are being used. By tracking interactions with new features, banks can make data-driven decisions on feature improvements and user experience enhancements.

## Redis Data Structures Used
- **Hashes**: To store user interaction data.
- **Streams**: To capture and process real-time user interactions.
- **Sorted Sets**: To rank features based on user engagement.


In [2]:
# Mock data for the use case
user_data = {
    'user_1001': {'feature': 'new_UI', 'interaction': 'click'},
    'user_1002': {'feature': 'biller_screen_enhancements', 'interaction': 'view'},
    'user_1003': {'feature': 'new_UI', 'interaction': 'click'},
    'user_1004': {'feature': 'new_card_reward_app', 'interaction': 'view'},
    'user_1005': {'feature': 'biller_screen_enhancements', 'interaction': 'click'},
}

# Use Hashes to store user interaction data
for user_id, interaction_info in user_data.items():
    r.hset(user_id, mapping=interaction_info)

# Use Streams to capture real-time user interactions
r.xadd('user_interactions', {'user_id': 'user_1001', 'feature': 'new_UI', 'interaction': 'click'})
r.xadd('user_interactions', {'user_id': 'user_1002', 'feature': 'biller_screen_enhancements', 'interaction': 'view'})

# Function to process user interactions and update engagement scores
def process_user_interactions():
    stream_entries = r.xrange('user_interactions')
    for entry in stream_entries:
        feature = entry[1]['feature']
        interaction = entry[1]['interaction']
        score = 1 if interaction == 'view' else 2  # Example scoring algorithm
        r.zincrby('feature_engagement', score, feature)

# Process the user interactions
process_user_interactions()

# Print the user interaction data
for user_id in user_data.keys():
    print(f"{user_id}: {r.hgetall(user_id)}")

# Print the feature engagement scores
feature_engagement = r.zrange('feature_engagement', 0, -1, withscores=True, desc=True)
print('Feature Engagement Scores:')
for feature in feature_engagement:
    print(feature)

user_1001: {'feature': 'new_UI', 'interaction': 'click'}
user_1002: {'feature': 'biller_screen_enhancements', 'interaction': 'view'}
user_1003: {'feature': 'new_UI', 'interaction': 'click'}
user_1004: {'feature': 'new_card_reward_app', 'interaction': 'view'}
user_1005: {'feature': 'biller_screen_enhancements', 'interaction': 'click'}
Feature Engagement Scores:
('new_UI', 2.0)
('biller_screen_enhancements', 1.0)
